## Importing all the libraries and dependencies

In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from chromedriver_py import binary_path 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import urllib.parse
from webdriver_manager.chrome import ChromeDriverManager

## Functionality to scrape the details of each job

In [51]:

def get_jobs_new(keyword, num_jobs, verbose):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    # Initializing the webdriver
    chromedriver_path = "C:/Program Files/chromedriver.exe"
    options = webdriver.ChromeOptions()
    service = webdriver.chrome.service.Service(executable_path=chromedriver_path)

    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')

    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1120, 1000)

    encoded_keyword = urllib.parse.quote(keyword)
    url = f"https://www.glassdoor.co.in/Job/united-stated-{encoded_keyword}-jobs-SRCH_IL.0,13_IN1_KO14,26.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword={encoded_keyword}&typedLocation=united%2520stated&context=Jobs&dropdown=0"
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  # If true, should still be looking for new jobs.

        # Let the page load. Change this number based on your internet speed.
        # Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(15)

        # Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CLASS_NAME, 'selected').click()
        except webdriver.common.exceptions.ElementClickInterceptedException:
            pass

        time.sleep(.1)
        
        #clicking on 'X' when sign up pops
        try:
            driver.find_element(By.XPATH, './/*[@id="LoginModal"]/div/div/div[2]/button').click()  # Clicking to the X.
        except NoSuchElementException:
            pass

        # Going through each job in this page
        job_list = driver.find_element(By.XPATH, './/*[@id="MainCol"]/div[1]/ul')  # ul for Job Listing. These are the buttons we're going to click.
        job_items = job_list.find_elements(By.TAG_NAME, "li")
        for job_item in job_items:
            try:
                #extracting the exact div for salary_estimate
                div_element = job_item.find_element(By.XPATH, './/div[4]')
                print("Progress: {}/{}".format(len(jobs), num_jobs))
                if len(jobs) >= num_jobs:
                    break

                job_item.click()
                time.sleep(1)
                collected_successfully = False

                while not collected_successfully:
                    try:
                        company_name = driver.find_element(By.XPATH, './/div[@class="css-87uc0g e1tk4kwz1" and @data-test="employerName"]').text
                        location = driver.find_element(By.XPATH, './/div[@class="css-56kyx5 e1tk4kwz5" and @data-test="location"]').text
                        job_title = driver.find_element(By.XPATH, './/div[@class="css-1vg6q84 e1tk4kwz4" and @data-test="jobTitle"]').text
                        job_description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
                        salary_estimate = div_element.text
                        collected_successfully = True
                    except webdriver.common.exceptions.NoSuchElementException:
                        #"Element not found. Retrying in 5 seconds..."
                        time.sleep(5)

                try:
                    rating = driver.find_element(By.XPATH, './/*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]/div/span').text
                except webdriver.common.exceptions.NoSuchElementException:
                    rating = -1

                if verbose:
                    print("Job Title: {}".format(job_title))
                    print("Salary Estimate: {}".format(salary_estimate))
                    print("Job Description: {}".format(job_description[:500]))
                    print("Rating: {}".format(rating))
                    print("Company Name: {}".format(company_name))
                    print("Location: {}".format(location))

                # Company Overview details
                try:
                    # <div class="d-flex justify-content-start css-rmzuhb e1pvx6aw0">
                    #    <span class="css-1taruhi e1pvx6aw1">Size</span>
                    #    <span class="css-i9gxme e1pvx6aw2">Unknown</span>
                    # </div>
                    size = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[1]/span[2]').text
                except webdriver.common.exceptions.NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[2]/span[2]').text
                except webdriver.common.exceptions.NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[3]/span[2]').text
                except webdriver.common.exceptions.NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[4]/span[2]').text
                except webdriver.common.exceptions.NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[5]/span[2]').text
                except webdriver.common.exceptions.NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[6]/span[2]').text
                except webdriver.common.exceptions.NoSuchElementException:
                    revenue = -1

                if verbose:
                    print("Size: {}".format(size))
                    print("Founded: {}".format(founded))
                    print("Type of Ownership: {}".format(type_of_ownership))
                    print("Industry: {}".format(industry))
                    print("Sector: {}".format(sector))
                    print("Revenue: {}".format(revenue))
                    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

                # Append all the data to jobs list as dictionaries
                jobs.append({
                    "Job Title": job_title,
                    "Salary Estimate": salary_estimate,
                    "Job Description": job_description,
                    "Rating": rating,
                    "Company Name": company_name,
                    "Location": location,
                    "Size": size,
                    "Founded": founded,
                    "Type of ownership": type_of_ownership,
                    "Industry": industry,
                    "Sector": sector,
                    "Revenue": revenue
                })
            #if there is some error while fetching data or any class name mismatches, we'll skip that row
            except:
                print("Error occurred while scraping a job. Skipping to the next job.")

        # Clicking on the next page
        try:
            driver.find_element(By.XPATH, './/*[@id="MainCol"]/div[2]/div/div[1]/button[7]').click()
        except webdriver.common.exceptions.NoSuchElementException:
            print("Scraping terminated before reaching the target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  # This line converts the dictionary object into a pandas DataFrame.


### Calling the function and printing dataset 

In [56]:
df = get_jobs_new('data science',500,False)
df

Progress: 0/500
Progress: 1/500
Progress: 2/500
Progress: 3/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 3/500
Progress: 4/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 4/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 4/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 4/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 4/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 4/500
Progress: 5/500
Progress: 6/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 6/500
Progress: 7/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 7/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 7/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 7/500
Progress: 8/500
Progress: 9/500
Error occurred while scraping a job. Skipping to t

Error occurred while scraping a job. Skipping to the next job.
Progress: 97/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 97/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 97/500
Progress: 98/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 98/500
Progress: 99/500
Progress: 100/500
Progress: 101/500
Progress: 102/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 102/500
Progress: 103/500
Progress: 104/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 104/500
Error occurred while scraping a job. Skipping to the next job.
Error occurred while scraping a job. Skipping to the next job.
Progress: 104/500
Progress: 105/500
Progress: 106/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 106/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 106/500
Error occurred while scraping a job. Skipping to the

Progress: 206/500
Progress: 207/500
Progress: 208/500
Progress: 209/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 209/500
Progress: 210/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 210/500
Progress: 211/500
Progress: 212/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 213/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 213/500
Progress: 214/500
Progress: 215/500
Progress: 216/500
Progress: 217/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 217/500
Progress: 218/500
Progress: 219/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 219/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 219/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 219/500
Progress: 220/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 220/500
Progress: 221/500
P

Progress: 340/500
Progress: 341/500
Progress: 342/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 342/500
Progress: 343/500
Progress: 344/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 345/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 345/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 345/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 345/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 345/500
Progress: 346/500
Progress: 347/500
Progress: 348/500
Progress: 349/500
Progress: 350/500
Progress: 351/500
Progress: 352/500
Progress: 353/500
Error occurred while scraping a job. Skipping to the next job.
Error occurred while scraping a job. Skipping to the next job.
Progress: 353/500
Progress: 354/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 354/500
Error occurred while scraping a job. 

Error occurred while scraping a job. Skipping to the next job.
Error occurred while scraping a job. Skipping to the next job.
Progress: 452/500
Progress: 453/500
Progress: 454/500
Progress: 455/500
Progress: 456/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 456/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 457/500
Progress: 458/500
Progress: 459/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 459/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 459/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 459/500
Progress: 460/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 460/500
Progress: 461/500
Error occurred while scraping a job. Skipping to the next job.
Progress: 461/500
Progress: 462/500
Progress: 463/500
Progress: 464/500
Progress: 465/500
Progress: 466/500
Progress: 467/500
Progress: 468/500
Error occurred whil

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Science,$1L - $2L (Employer Est.),Company Overview: Pueo is known for bringing t...,4.9,Pueo Business Solutions\n4.9,"Quantico, VA",51 to 200 Employees,2017,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD)
1,Data Science Analyst,$75T - $1L (Employer Est.),Claritas Rx is a digital health venture that b...,4.9,Claritas Rx Inc.\n4.9,"South San Francisco, CA",51 to 200 Employees,2011,Company - Private,Biotech & Pharmaceuticals,Pharmaceutical & Biotechnology,Unknown / Non-Applicable
2,Data Scientist,Easy Apply,Anika Systems is looking for a passionate and ...,4.5,Anika Systems\n4.5,Remote,51 to 200 Employees,2005,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD)
3,Lead Data Scientist,Easy Apply,Senior/Lead Data Scientist\nFinchAI is a fast-...,3.3,Finch Computing\n3.3,Remote,1 to 50 Employees,2014,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable
4,Junior Data Scientist,$80T - $90T (Employer Est.),"As a Junior Data Scientist, you will help supp...",3.5,"SoundThinking, Inc.\n3.5","Fremont, CA",201 to 500 Employees,1996,Company - Public,Software Development,Information Technology,$25 to $50 million (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...
495,"Manager, CRM Data Science Analytics",$2L (Employer Est.),Job ID: 225743\nLocation Name: FSC REMOTE SF/N...,3.6,Sephora\n3.6,"San Francisco, CA",10000+ Employees,1969,Company - Private,Beauty & Personal Accessories Stores,Retail & Wholesale,$2 to $5 billion (USD)
496,Junior Data Scientist,$80T - $90T (Employer Est.),"As a Junior Data Scientist, you will help supp...",3.5,"SoundThinking, Inc.\n3.5","Fremont, CA",201 to 500 Employees,1996,Company - Public,Software Development,Information Technology,$25 to $50 million (USD)
497,Data Scientist - Data Science and Engineering ...,$1L - $2L (Employer Est.),Career Category\nInformation Systems\nJob Desc...,4.1,Amgen\n4.1,"Los Angeles, CA",10000+ Employees,1980,Company - Public,Biotech & Pharmaceuticals,Pharmaceutical & Biotechnology,$10+ billion (USD)
498,Data Science Admissions Specialist (Part-time),$18.00 Per hour (Employer Est.),What is Pathrise?\nPathrise (YC W18) connects ...,4.7,Pathrise\n4.7,Remote,51 to 200 Employees,2017,Company - Private,Education Support & Training Services,Education,$1 to $5 million (USD)


### Saving as csv file

In [58]:
df_copy = df
#saving the dataframe to a csv file
df.to_csv('ds_salary_data.csv')